# EMS ROUTING

In [1]:
import datetime
import folium
import json
import numpy as np
import requests

from h3 import h3
from IPython.core.display import HTML
from shapely import wkt, geometry

In [2]:
tile = 'https://api.mapbox.com/styles/v1/mapbox/streets-v10/tiles/256/{z}/{x}/{y}?access_token=pk.eyJ1IjoiZGl2OSIsImEiOiJjam9jZWo5eGYwbW0zM3BydHNnbm5wbGs5In0.VRTb28kY6k_5WObVO6-QZg'
m = folium.Map(location=[42.4485, -76.4804],
               zoom_start=13,
               detect_retina=True)

m.add_tile_layer(tiles=tile,
                 attr='Map data &copy; <a href="http://openstreetmap.org">OpenStreetMap</a> contributors, ' +
                      '<a href="http://creativecommons.org/licenses/by-sa/2.0/">CC-BY-SA</a>, ' +
                      'Imagery © <a href="http://mapbox.com">Mapbox</a>',
                 name="Mapbox Street",
                 detect_retina=True)
# 
style = lambda x: {'fillColor': '#100000', 'fillOpacity':0}
 
with open('ithaca.json','r') as f:
    geoJson = json.load(f)
    
geo = folium.GeoJson(
    geoJson,
    name='boundary',
    style_function=style
).add_to(m)

In [3]:
# Resolution goes from 0..15, with 15 being the max
res = 10

hexagons = list(h3.polyfill(geoJson,res, True))
edge = h3.edge_length(res) * 1e3

print("Num Hexagons:", len(hexagons))
print("Edge Length (m):", edge)

Num Hexagons: 1000
Edge Length (m): 65.907807


In [4]:
def grid_style(feature):
    return {
        'fillOpacity': 0.3,
        'color': 'red'
    }

# Get hexagon coordinates
coords = list(map(lambda x: [h3.h3_to_geo_boundary(x, True)], hexagons))
gridJson = {}
geom = {}
geom["type"] = "MultiPolygon"
geom["coordinates"] = coords

gridJson["type"] = "Feature"
gridJson["geometry"] = geom

# Plot Grid
folium.GeoJson(
    gridJson,
    name='grid',
    style_function=grid_style
    ).add_to(m)

# Add Parking Markers
park_spots = np.loadtxt('parking.txt')

# Create a Folium feature group for this layer
pt_lyr = folium.FeatureGroup(name = 'parking spots')
for spot in park_spots:
    pt_lyr.add_child(folium.Marker(location=spot, icon=folium.Icon(icon='product-hunt', prefix='fa')))
m.add_child(pt_lyr)

folium.LayerControl().add_to(m)

m.save('routing.html')
m

In [5]:
#dep_pts should be an 1x2 array of a lat/long pair
#reference: 
#http://docs.traveltimeplatform.com/reference/time-map/
def gen_data(dep):
    curr_time = datetime.datetime.now().isoformat()
    d = {}
    d['departure_searches'] = [{'id' : 'dep', 'coords' : {'lat' : dep[0], 'lng' : dep[1]},
                               "transportation": {"type": "driving"},
                               "departure_time": curr_time,
                               "travel_time": 360 }]  
    return d

def make_request(point):
    host = 'http://api.traveltimeapp.com/v4/time-map'
    app_id = '41a5ea7b'
    tt_api_key = '907b7ed3de1d6b08e6713ce05e2a6041'

    head = {"Content-Type": "application/json", "Accept": "application/vnd.wkt+json",
            "X-Application-Id": app_id, "X-Api-Key": tt_api_key}
    r = requests.post(host, data=json.dumps(gen_data(point)), headers=head)
    if r:
        return r.json()
    else:
        print(r.status_code)
        display(HTML(r.text))

In [6]:
def get_isochrone(point):
    res = make_request(point)
    iso = res['results'][0]['shape']
    return iso

In [7]:
# iso_json = {}
# poly_lst = []
# for spot in park_spots:
#     iso = get_isochrone(spot)
#     poly_lst.append(iso)
    
# iso_json['polygons'] = poly_lst

# with open("isochrone.json", "w") as f:
#     json.dump(iso_json, f)

In [8]:
with open("isochrone.json", "r") as f:
    isochrones = json.load(f)

polygons = []
for iso in isochrones['polygons']:
    iso_geom = wkt.loads(iso) # load geometry from WKT string
    polygons.append(iso_geom)
    iso_coords = list(iso_geom.exterior.coords) # get coords from exterior ring
    iso_coords = [(y,x) for x,y in iso_coords] # swap (x,y) to (y,x)

    folium.Polygon(iso_coords,
                   color='green',
                   fill_color='green',
                   fill_opacity=0.1,
                   ).add_to(m)
    
m.save('isochrone.html')
m

In [9]:
# Create Graph

inf = 10
edges = [] # Start, End, Capacities

n = len(polygons)
for i, poly in enumerate(polygons):
    for j, h in enumerate(hexagons):
        point = h3.h3_to_geo(h)
        if poly.contains(geometry.Point(point[::-1])):
            edge = [i, n+j, inf]
            edges.append(edge)

In [10]:
len(edges)

9033

In [11]:
# hexagons = sorted(hexagons, key=lambda hexa: h3.h3_to_geo(hexa)[0]-h3.h3_to_geo(hexa)[1])

with open('density.json', 'r') as f:  
    hexCoords = json.load(f)

hexCoords = np.array(hexCoords)
hexagons = list(hexCoords[:,0])
den = hexCoords[:,1]

# source and sink indices
_SOURCE = len(polygons)+len(hexagons)
_SINK = _SOURCE+1
            
# source to parking lots
for i, poly in enumerate(polygons):
    edge = [_SOURCE, i, inf]
    edges.append(edge)
print(len(edges))

# parking lots to sink
for j,h in enumerate(hexagons):
    edge = [n+j, _SINK, den[j]]
    edges.append(edge)
print(len(edges))

9055
10055
